In [2]:
#importing Libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

* pandas to create dataframe
* BeautifulSoup to do web scraping

In [3]:
#the url of the first page
url = "https://www.imdb.com/list/ls068082370/"

In [4]:
def all_page_link(start_url):
    all_urls = []
    url = start_url
    while(url != None):            
        all_urls.append(url)
        soup = BeautifulSoup(requests.get(url).text,"html.parser")
        next_links = soup.find_all(class_='flat-button lister-page-next next-page')    
        if (len(next_links) == 0):         
            url = None
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
            url = next_page
    return all_urls

director_and_actor is a function used to separate director and actor, as they are present in the same string 

In [5]:
def director_and_actor(Director_and_star):
    Director_and_star =  Director_and_star.replace("\n","")
    Director_and_star = Director_and_star.replace("|","")
    Director_and_star = Director_and_star.split("Stars:")
    Director_and_star[0] = Director_and_star[0].replace("Director:","")
    Director_and_star[0] = Director_and_star[0].replace("Directors:","")
    for i in range(10):
        Director_and_star[0]=Director_and_star[0].replace("  "," ")
    director = Director_and_star[0][1:]
    stars = Director_and_star[1]
    stars = stars.replace(":","")
    return director,stars

Function to separate votes and gross, as they are present in the same string

In [6]:
def votes_and_gross_conveter(votes_and_gross):
    votes_and_gross_list = []    
    for i in votes_and_gross:
        votes_and_gross_list.append(i.text)
    if(len(votes_and_gross)==2):
        votes=votes_and_gross_list[0]
        gross = votes_and_gross_list[1]
    else:
        votes=votes_and_gross_list[0]
        gross = None
    
    return votes,gross

* "main_array" = stores all the big data.!
* "id" = stores the id number of a movie.
* "name" = Stores the name of the movie.
* "year" = Stores the year in which the movie was released.
* "run_time" = Stores the total runtime in minutes.
* "genre" = Stores the type of movie.
* "rating" = Stores the IMDb rating of the movie.
* "about" = Stores a short discription about the movie.
* "director" = stores the name of the director.
* "actors" = Stores the name of the director.
* "votes" = Stores the number of votes of the movie.
* "gross" = Stores the net income of the movie.

In [7]:
main_array = []
for url in tqdm(all_page_link("https://www.imdb.com/list/ls068082370/")):     #Runs the function for all the pages.
    soup = BeautifulSoup(requests.get(url).text,"html.parser")         #Extracts out the main html code.
    for link in soup.find_all(class_='lister-item-content'):
        id = int(link.find('span',{"class":"lister-item-index unbold text-primary"}).text[:-1])
        name = link.find('a').text
        year = link.find('span',{"class":"lister-item-year text-muted unbold"}).text[1:5]
        run_time = link.find('span',{"class":"runtime"}).text
        genre = link.find('span',{"class":"genre"}).text[1:]
        rating = link.find('span',{"class":"ipl-rating-star__rating"}).text
        about = link.find_all('p')[1].text[5:]
        director,actors = director_and_actor(link.find_all('p',{"class":"text-muted text-small"})[1].text)
        votes, gross = votes_and_gross_conveter(link.find_all('span',{"name":"nv"}))
        votes = int(votes.replace(",",""))
        list_of_all = [id,name,year,run_time,genre,rating,about,director,actors,votes,gross]
        main_array.append(list_of_all)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:31<00:00, 10.68s/it]


In [8]:
#this index variable contains the name of the columns of the data frame.
index = ["id","name","year","run_time","genre","rating","about","director","actors","votes","gross"]

In [9]:
df = pd.DataFrame(main_array,columns=index)   #creating the DataFrame using "main_array"

In [10]:
df

,id,name,year,run_time,genre,rating,about,director,actors,votes,gross
0,1,The Shawshank Redemption,1994,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",2123733,$28.34M
1,2,The Godfather,1972,175 min,"Crime, Drama",9.2,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1458140,$134.97M
2,3,The Dark Knight,2008,152 min,"Action, Crime, Drama",9,When the menace known as The Joker emerges fro...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2088869,$534.86M
3,4,The Godfather: Part II,1974,202 min,"Crime, Drama",9,The early life and career of Vito Corleone in ...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...",1014144,$57.30M
4,5,Pulp Fiction,1994,154 min,"Crime, Drama",8.9,"The lives of two mob hitmen, a boxer, a gangst...",Quentin Tarantino,"John Travolta, Uma Thurman, Samuel L. Jackson,...",1663444,$107.93M
5,6,Schindler's List,1993,195 min,"Biography, Drama, History",8.9,"In German-occupied Poland during World War II,...",Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley, Caro...",1102012,$96.90M
6,7,The Lord of the Rings: The Return of the King,2003,201 min,"Adventure, Drama, Fantasy",8.9,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",1511063,$377.85M
7,8,12 Angry Men,1957,96 min,Drama,8.9,A jury holdout attempts to prevent a miscarria...,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam, John ...",604813,$4.36M
8,9,"Il buono, il brutto, il cattivo",1966,161 min,Western,8.8,A bounty hunting scam joins two men in an unea...,Sergio Leone,"Clint Eastwood, Eli Wallach, Lee Van Cleef, Al...",630845,$6.10M
9,10,Forrest Gump,1994,142 min,"Drama, Romance",8.8,"The presidencies of Kennedy and Johnson, the e...",Robert Zemeckis,"Tom Hanks, Robin Wright, Gary Sinise, Sally Field",1633113,$330.25M


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
id          250 non-null int64
name        250 non-null object
year        250 non-null object
run_time    250 non-null object
genre       250 non-null object
rating      250 non-null object
about       250 non-null object
director    250 non-null object
actors      250 non-null object
votes       250 non-null int64
gross       218 non-null object
dtypes: int64(2), object(9)
memory usage: 21.6+ KB


In [14]:
df.shape

(250, 11)

In [15]:
df.columns


Index(['id', 'name', 'year', 'run_time', 'genre', 'rating', 'about',
       'director', 'actors', 'votes', 'gross'],
      dtype='object')